# Week 3 Day 1

Welcome to Google Colab - using free and powerful compute in the cloud.

Google Colab gives you a remote Notebook-style browser window on to a machine, and you run code "locally" on that machine.

## Benefits of Colab

1. Free access to T4 GPUs!
2. Easy ability to share code and collaborate on it
3. Everyone gets to use identical code - no environment differences

## Downsides of Colab

1. As it's free, Google reserves the right to bump you off the box at any point ("reset the runtime"), and this happens most quickly if no code is running. They can also downgrade you from a T4 to a CPU-only box. Sometimes this happens silently. You need to start everything again from the top. Paid plans last longer.
2. You need to pip install packages every single time (but no pip installs today)
3. There's some latency - it's not as interactive as coding on your own box

## Survival Guide

1. Always start by pressing the drop down arrow by Connect on the top right, and "Connect to a hosted runtime: T4"
2. From that dropdown, "View Resources" to check you have a GPU and monitor memory
3. If things go awry, Runtime >> Disconnect and Delete Runtime, Connect again, and then run cells from the top
4. Always run the pip installs! Ignore pip dependency errors.

Runtime >> Restart session: this restarts the Python Kernel, but pip packages remain installed and the disk remains the same.

Runtime >> Diconnect and delete session. This wipes everything.

In [11]:
# !uv pip install -q --upgrade transformers==4.56.2
# !uv pip install diffusers

In [4]:
import torch
print("¿CUDA disponible?", torch.cuda.is_available())
print("Dispositivo actual:", torch.cuda.current_device())
print("Nombre de la GPU:", torch.cuda.get_device_name(0))

¿CUDA disponible? True
Dispositivo actual: 0
Nombre de la GPU: NVIDIA GeForce RTX 4060 Ti


In [5]:
# Let's check the GPU - it should be a Tesla T4 if yyou are working with colab

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('Tesla T4') >= 0:
    print("Success - Connected to a T4")
  else:
    print("NOT CONNECTED TO A T4")

Wed Nov 26 12:43:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.64.01              Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   43C    P5             27W /  165W |    1454MiB /  16380MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Connecting Hugging Face

You'll need to log in to the HuggingFace hub if you've not done so before.

1. If you haven't already done so, create a **free** HuggingFace account at https://huggingface.co and navigate to Settings from the user menu on the top right. Then Create a new API token, giving yourself write permissions.  

**IMPORTANT** when you create your HuggingFace API key, please be sure to select WRITE permissions for your key by clicking on the WRITE tab, otherwise you may get problems later. Not "fine-grained" but "write".

2. Back here in colab, press the "key" icon on the side panel to the left, and add a new secret:  
  In the name field put `HF_TOKEN`  
  In the value field put your actual token: `hf_...`  
  Ensure the notebook access switch is turned ON.

3. Execute the cell below to log in. You'll need to do this on each of your colabs. It's a really useful way to manage your secrets without needing to type them into colab.

In [12]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# from google.colab import userdata
load_dotenv(override=True)
hf_token = os.getenv('HF_TOKEN')

# hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
from IPython.display import display
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", dtype=torch.float16, variant="fp16")
pipe.to("cuda")
prompt = "A class of students learning AI engineering in a vibrant pop-art style"
image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0.0).images[0]
display(image)

In [14]:
# Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
from IPython.display import display
from diffusers import DiffusionPipeline
import torch

model_name = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(model_name,
                                         dtype=torch.float16,
                                         use_safetensors=True,
                                         variant="fp16")
pipe.to("cuda")

prompt = "A class of data scientists learning AI engineering in a vibrant high-energy pop-art style"

image = pipe(prompt=prompt, num_inference_steps=30).images[0]

display(image)


In [ ]:
# Restart the kernel

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
                                         dtype=torch.float16,
                                         variant="fp16",
                                         use_safetensors=True)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0",
                                            text_encoder_2=base.text_encoder_2,
                                            vae=base.vae,
                                            dtype=torch.float16,
                                            use_safetensors=True,
                                            variant="fp16",)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A class of data scientists learning AI engineering in a vibrant high-energy pop-art style"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

display(image)

Este código implementa **la inferencia en dos etapas del modelo Stable Diffusion XL (SDXL)** usando el **pipeline base + refiner**, una estrategia optimizada para generar imágenes de alta calidad con menor costo computacional.

Vamos a desglosarlo paso a paso:

---

### 🔧 **1. Cargar el pipeline base**
```python
base = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
                                         torch_dtype=torch.float16,
                                         variant="fp16",
                                         use_safetensors=True)
base.to("cuda")
```
- Carga el **modelo base de SDXL**, que es el encargado de generar la imagen desde ruido.
- Usa **precisión mixta** (`float16`) para ahorrar memoria y acelerar la inferencia.
- `variant="fp16"` indica que se descarguen los pesos en formato FP16 (más ligero).
- `use_safetensors=True` usa el formato seguro y rápido `safetensors` en lugar de `pickle`.
- Mueve el modelo a la GPU con `.to("cuda")`.

---

### 🔧 **2. Cargar el pipeline refiner**
```python
refiner = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0",
                                            text_encoder_2=base.text_encoder_2,
                                            vae=base.vae,
                                            torch_dtype=torch.float16,
                                            use_safetensors=True,
                                            variant="fp16")
refiner.to("cuda")
```
- Carga el **modelo refinador de SDXL**, que no genera desde cero, sino que **mejora una imagen parcial ya generada**.
- **Reutiliza componentes del modelo base**:
  - `text_encoder_2`: SDXL usa dos text encoders; el segundo ya está cargado en `base`, así que lo comparte (ahorra RAM y evita duplicados).
  - `vae`: el autoencoder variacional (que decodifica latentes a píxeles) también se comparte.
- Esto es clave: el refiner **no necesita su propio VAE ni su propio text encoder 2**, ya que opera sobre la misma representación latente y semántica que el base.

---

### ⚙️ **3. Configurar la estrategia de inferencia en dos etapas**
```python
n_steps = 40
high_noise_frac = 0.8
```
- **40 pasos de denoising en total**.
- **El 80% del proceso (32 pasos)** lo hace el **modelo base**.
- **El 20% final (8 pasos)** lo hace el **refiner**.

Esto se basa en la observación de que:
> Los primeros pasos de denoising definen la composición general, mientras que los últimos refinan detalles finos (texturas, bordes, coherencia local).  
> El **refiner está entrenado específicamente para mejorar esos últimos pasos**, y no necesita ver todo el ruido inicial.

---

### 🖼️ **4. Primera etapa: generación parcial con el modelo base**
```python
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,    # ← Detiene en el 80%
    output_type="latent",             # ← Devuelve latentes, no píxeles
).images
```
- Genera una imagen, pero **solo hasta el 80% del proceso**.
- `output_type="latent"`: devuelve la representación **latente** (un tensor de baja resolución en el espacio del VAE), **no una imagen en píxeles**.
- Esto es eficiente: no se decodifica a RGB innecesariamente.

---

### 🖼️ **5. Segunda etapa: refinamiento con el modelo refiner**
```python
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,  # ← Empieza donde terminó el base
    image=image,                      # ← Recibe los latentes del base
).images[0]
```
- El refiner **continúa el denoising desde el 80% hasta el 100%**.
- Recibe los **latentes intermedios** del base y los mejora.
- Finalmente, **decodifica a una imagen RGB** (porque `output_type` no se especifica, usa el valor predeterminado: `"pil"`).

---

### ✅ **¿Por qué hacer esto?**
- **Mayor calidad visual**: el refiner mejora coherencia, texturas y detalles finos.
- **Eficiencia**: el refiner no necesita procesar los pasos iniciales (más costosos en términos de estructura global).
- **Ahorro de memoria**: al compartir `text_encoder_2` y `vae`, se evita cargar duplicados.
- **SDXL fue diseñado así**: la arquitectura oficial de Stability AI usa esta estrategia en producción.

---

### ⚠️ Notas importantes
- **Ambos modelos deben estar en la GPU** (`.to("cuda")`).
- **El prompt debe ser el mismo en ambas etapas** para coherencia.
- Puedes ajustar `high_noise_frac`:
  - Valores más altos (ej. `0.8–0.9`) → más trabajo para el base → más rápido pero menos refinamiento.
  - Valores más bajos (ej. `0.6`) → más refinamiento, pero el refiner recibe una imagen menos definida.

---

### 📌 En resumen:
Este código **genera una imagen con SDXL usando dos modelos especializados**:  
1. **Base**: crea la imagen desde el ruido hasta un estado intermedio.  
2. **Refiner**: toma ese estado intermedio y lo mejora en los pasos finales.  

Es la forma **recomendada y óptima** de usar Stable Diffusion XL para alta calidad.

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [2]:
# !uv pip install --upgrade datasets==3.6.0

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import torch
from IPython.display import Audio

synthesiser = pipeline("text-to-speech",
                       "microsoft/speecht5_tts",
                       device='cuda')

embeddings_dataset = load_dataset("matthijs/cmu-arctic-xvectors", split="validation", trust_remote_code=True)
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
speech = synthesiser("Hi to an artificial intelligence engineer, on the way to mastery!", forward_params={"speaker_embeddings": speaker_embedding})

Audio(speech["audio"], rate=speech["sampling_rate"])

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## This next cells will only work on a powerful GPU box like an A100

This is not available on a free T4 box.

I just want to show off what's possible with a small paid budget..

Rough pricing:

- $9.99 = 100 compute units
- An A100 = 5.37 compute units per hour as of Oct 2025 (for me)

In [ ]:
# Let's check the GPU - it should be an A100

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('A100') >= 0:
    print("Success - Connected to an NVIDIA A100")
  else:
    print("NOT CONNECTED TO AN A100")

In [ ]:
import torch
from diffusers import FluxPipeline
from IPython.display import display
from datetime import datetime

start = datetime.now()

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16).to("cuda")
generator = torch.Generator(device="cuda").manual_seed(0)
prompt = "A class of data scientists learning AI engineering in a vibrant high-energy pop-art style"

# Generate the image using the GPU
image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    generator=generator
).images[0]

display(image)

stop = datetime.now()


In [ ]:
# Cost estimate

seconds = (stop-start).total_seconds()
units_per_hour = 5.37
estimated_units = (5.37 / 3600) * seconds
estimated_cost = estimated_units * (9.99/100)
print(f"This took {seconds:.1f} seconds and cost an estimated ${estimated_cost:.3f}")

# But there's a catch - you pay for all the time the kernel is active, not just while it's actually calculating!
# So remember to shut down a Paid kernel..